In [ ]:
import pandas as pd
import json
import requests
import folium

In [ ]:
"""añadir la base de datos de bigquery que estan todas las ciudades"""
data = {'Neighbourhoods':  ['Aalborg', 'Nørresundby', 'Svenstrup', 'Nibe',   'Klarup', 'Vodskov', 'Hasseris', 'Skalborg',    'Vejgaard'],
        'Latitude': [       '57.0472', '57.05877', '    56.98',   '56.9833', '57.0167', '57.1',  '57.0396822', '57.0084631', '57.0407625'],
        'Longitude': [      '9.9201',  '9.92284',      '9.9833',  '9.6333',   '10.05', '10.0333', '9.8855685', '9.8858048',   '9.951511']
       }

df = pd.DataFrame (data, columns = ['Neighbourhoods','Latitude', 'Longitude'])
df

,Neighbourhoods,Latitude,Longitude
0,Aalborg,57.0472,9.9201
1,Nørresundby,57.05877,9.92284
2,Svenstrup,56.98,9.9833
3,Nibe,56.9833,9.6333
4,Klarup,57.0167,10.05
5,Vodskov,57.1,10.0333
6,Hasseris,57.0396822,9.8855685
7,Skalborg,57.0084631,9.8858048
8,Vejgaard,57.0407625,9.951511


In [ ]:


CLIENT_ID = '***' #clienteid de la empresa
CLIENT_SECRET = '****' #cliente secreto de la empresa
VERSION = 20202808
radius = 100000 #radio a mirar
LIMIT = 100000 #limte total
#CATEGORIA = 'restaurant' #categoría de la que se quiere buscar

In [ ]:
def es_repetido(anadido, id):
  for v in anadido:
    if v[0][6] == id:
      return True
      break
  return False

In [ ]:
def getNearbyVenues(names, latitudes, longitudes):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)


        results = requests.get(url).json()["response"]['groups'][0]['items']
        for v in results:
            venues_list.append([(
                name,
                lat,
                lng,
                v['venue']['name'],
                v['venue']['location']['lat'],
                v['venue']['location']['lng'],
                v['venue']['id'],
                v['venue']['categories'][0]['name'])])

    return (venues_list)

In [ ]:
antwerp_venues = getNearbyVenues(names = df['Neighbourhoods'],
                              latitudes = df['Latitude'],
                            longitudes = df['Longitude'])

Aalborg
Nørresundby
Svenstrup
Nibe
Klarup
Vodskov
Hasseris
Skalborg
Vejgaard


In [ ]:
antwerp_venues

[[('Aalborg',
   '57.0472',
   '9.9201',
   'Utzon Center',
   57.04943462191629,
   9.92635667324066,
   '4bc5652e693695210a128488',
   'Cultural Center')],
 [('Aalborg',
   '57.0472',
   '9.9201',
   'Behag Din Smag',
   57.047945419999266,
   9.924835194765956,
   '53d35946498ecbe059bff679',
   'Café')],
 [('Aalborg',
   '57.0472',
   '9.9201',
   'Havnefronten Aalborg',
   57.05073179213265,
   9.922499770249889,
   '4bf57155ff90c9b681615628',
   'Waterfront')],
 [('Aalborg',
   '57.0472',
   '9.9201',
   'The Irish House',
   57.04966493996621,
   9.921952152270608,
   '4bb3989542959c742a2b222c',
   'Irish Pub')],
 [('Aalborg',
   '57.0472',
   '9.9201',
   'Cafe Vi.2',
   57.04903582795331,
   9.919848288972291,
   '4bade994f964a52010723be3',
   'Café')],
 [('Aalborg',
   '57.0472',
   '9.9201',
   'Restaurant Rusk',
   57.04642144209802,
   9.920660525548742,
   '4ceff3767b943704f9243653',
   'French Restaurant')],
 [('Aalborg',
   '57.0472',
   '9.9201',
   'Bistro59',
   57.04

In [ ]:
def eliminar_duplicados(antwerp_venues):
  unicos = []
  unicos.append(antwerp_venues[0])
  for v in antwerp_venues:
    if es_repetido(unicos, v[0][6]) == False:
            unicos.append(v);
  return unicos


In [ ]:
unicos = eliminar_duplicados(antwerp_venues)

In [ ]:
def filtrar_category(unicos, category):
  filtrados = []
  for v in unicos:
    if category in v[0][7]:
      filtrados.append(v)
  return filtrados

In [ ]:
"""meter las categorias que hay en big query"""
print("¿Que categoría desea filtrar?")
print("Recuerda que la primera letra debe estar en mayúscula")
category = input()
print(f"Datos filtrados por la categoría, {category}")

¿Que categoría desea filtrar?
Recuerda que la primera letra debe estar en mayúscula
Restaurant
Datos filtrados por la categoría, Restaurant


In [ ]:
unicos_filtrados = filtrar_category(unicos, category)

In [ ]:
nearby_venues = pd.DataFrame([item for venue_list in unicos_filtrados for item in venue_list])


In [ ]:
nearby_venues

,0,1,2,3,4,5,6,7
0,Aalborg,57.0472,9.9201,Restaurant Rusk,57.046421,9.920661,4ceff3767b943704f9243653,French Restaurant
1,Aalborg,57.0472,9.9201,Bistro59,57.044078,9.920909,54d26d4e498e0840029c88d2,French Restaurant
2,Aalborg,57.0472,9.9201,Karma Sushi,57.050867,9.919225,4ba773e6f964a520199539e3,Sushi Restaurant
3,Aalborg,57.0472,9.9201,Mortens Kro,57.045913,9.921989,4b6acc8ff964a520dfe02be3,Scandinavian Restaurant
4,Aalborg,57.0472,9.9201,Jin Shing,57.067567,9.943648,4bcdb3c0b6c49c7424bd9591,Chinese Restaurant
5,Aalborg,57.0472,9.9201,Azzurra,57.049100,9.914895,4bf0322dc8d920a1008d9430,Italian Restaurant
6,Aalborg,57.0472,9.9201,La Locanda,57.048711,9.920353,5564c05b498e887e7dbe3000,Italian Restaurant
7,Aalborg,57.0472,9.9201,Hos Isidor Henius,57.048991,9.926096,54425eaf498ee3b940f3427d,Scandinavian Restaurant
8,Aalborg,57.0472,9.9201,Hirtshals Fiskehus,57.591743,9.956101,51af6539498e52c3ea9f1962,Seafood Restaurant
9,Aalborg,57.0472,9.9201,Jerry's Restaurant & Bryghus,57.438723,10.536196,4d6be1863a6bf04dfcce587d,American Restaurant


In [ ]:
nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'venue ID',
                  'Venue Category']

In [ ]:
nearby_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,venue ID,Venue Category
0,Aalborg,57.0472,9.9201,Restaurant Rusk,57.046421,9.920661,4ceff3767b943704f9243653,French Restaurant
1,Aalborg,57.0472,9.9201,Bistro59,57.044078,9.920909,54d26d4e498e0840029c88d2,French Restaurant
2,Aalborg,57.0472,9.9201,Karma Sushi,57.050867,9.919225,4ba773e6f964a520199539e3,Sushi Restaurant
3,Aalborg,57.0472,9.9201,Mortens Kro,57.045913,9.921989,4b6acc8ff964a520dfe02be3,Scandinavian Restaurant
4,Aalborg,57.0472,9.9201,Jin Shing,57.067567,9.943648,4bcdb3c0b6c49c7424bd9591,Chinese Restaurant
5,Aalborg,57.0472,9.9201,Azzurra,57.049100,9.914895,4bf0322dc8d920a1008d9430,Italian Restaurant
6,Aalborg,57.0472,9.9201,La Locanda,57.048711,9.920353,5564c05b498e887e7dbe3000,Italian Restaurant
7,Aalborg,57.0472,9.9201,Hos Isidor Henius,57.048991,9.926096,54425eaf498ee3b940f3427d,Scandinavian Restaurant
8,Aalborg,57.0472,9.9201,Hirtshals Fiskehus,57.591743,9.956101,51af6539498e52c3ea9f1962,Seafood Restaurant
9,Aalborg,57.0472,9.9201,Jerry's Restaurant & Bryghus,57.438723,10.536196,4d6be1863a6bf04dfcce587d,American Restaurant


In [ ]:
nearby_venues.to_excel('aalborg.xlsx', sheet_name='aalborg_scrape')